<a href="https://colab.research.google.com/github/PHaasbroek/Google-Colab/blob/main/Fluid-Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The aim of this notebook is to simulate a simple numerical domain.

1. Implement a laminar, steady, fluid solver for one dimensional pipe flow.
2. Implement a RANS turbulence model.
  1. Employ the turbulence model in the one dimensional pipe flow.
  2. Employ the turbulence model for flow past a backward facing step.
  3. Employ the turbulence model for flow past an aerofoil.

# The Navier-Stokes Equations

The incompressible Navier-Stokes (NS) equations for three dimensions are given as:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$

$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} + w \frac{\partial v}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left(\frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} + \frac{\partial^2 v}{\partial z^2}\right)$$

$$\frac{\partial w}{\partial t} + u \frac{\partial w}{\partial x} + v \frac{\partial w}{\partial y} + w \frac{\partial w}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial z} + \nu \left(\frac{\partial^2 w}{\partial x^2} + \frac{\partial^2 w}{\partial y^2} + \frac{\partial^2 w}{\partial z^2}\right)$$


$$\frac{\partial p}{\partial x} + \frac{\partial p}{\partial y} + \frac{\partial p}{\partial y} = 0$$

where:

* $u$ is the velocity in the $x$-direction
* $v$ is the velocity in the $y$-direction
* $w$ is the velocity in the $z$-direction
* $p$ is the pressure
* $\rho$ is the density
* $\nu$ is the kinematic viscosity

# Solution to One-Dimensional Pipe Flow
## Navier-Stokes equations
For the first (1.) case, i.e. one dimensional flow through a pipe, the NS equation is reduced to:
$$ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$
$$\frac{\partial u}{\partial t} = 0$$

The dimensionless first cell height, $y_1^+$, is defined as
$$y_1^+ = \frac{\rho U_\tau y_1}{\mu},$$
where $\rho$ is the fluid density, $U_\tau$ is the friction velocity, $y_1$ is the first cell height, and $\mu$ is the fluid dynamic viscosity. Friction velocity, $U_\tau$ is given as
$$U_\tau = \sqrt{\frac{\tau_w}{\rho}}, $$ where wall shear stress, $\tau_w$, is given as
$$\tau_w = \frac{1}{2} C_f \rho U^2.$$
Skin friction, $C_f$, is approximated empirically as
$$C_f = 0.079 Re^{-0.25}$$
for internal flow, and as
$$C_f = 0.058 Re^{-0.2}$$
for external flow.


In [3]:
import math

cellNumberHeight = 100 # number of cells through the height of the domain
cellWidth = 1.0 # unit width of cells
firstCellHeight = 8e-5 # [m]
diameter = 0.050 # [m]
meanVelocity = 2.0 # [m/s]
density = 1000.0 # [kg/m^3]
dynamicViscosity = 0.01 # [Pa * s]

Re = diameter * meanVelocity * density / dynamicViscosity
skinFriction = 0.079 * Re**(-0.25)
wallShearStress = 0.5 * skinFriction * density * meanVelocity**2
frictionVelocity = math.sqrt(wallShearStress / density)
dimensionlessFirstCellHeight = density * frictionVelocity * firstCellHeight / dynamicViscosity #

print("Reynolds number: " + str(Re) + " [-]")
print("Skin friction: " + str(skinFriction) + " [-]")
print("Wall shear stress: " + str(wallShearStress) + " [Pa]")
print("Friction velocity: " + str(frictionVelocity) + " [m/s]")
print("Dimensionless first cell height: " + str(dimensionlessFirstCellHeight) + " [-]")


Reynolds number: 10000.0 [-]
Skin friction: 0.0079 [-]
Wall shear stress: 15.8 [Pa]
Friction velocity: 0.12569805089976535 [m/s]
Dimensionless first cell height: 1.005584407198123 [-]


## Mesh Generation
The mesh is created by:
1. Assume the first cell height is as specified.
2. Assume the number of cells are as specified.
3. Assume constant expansion ratio.
3. Calculating the expansion ratio from the given specifications.



In [39]:
# NonlinearSolver settings
meshExpansionRatiox0 = 5.0 # initial guess for nonlinearSolver
meshExpansionRatioTol = 1e-12
meshExpansionRatioMaxIter = 1000

def sumOfGeometricSeries_f(firstEntry = 1.0, ratio = 1.0, numExpandingCells = 1):
  # assume that the there will always be a difference in cell height
  if ratio==1.0:
    ratio += 1e-9

  return firstEntry * (1.0 - ratio ** numExpandingCells) / (1.0 - ratio)


# prompt: Generate  a loop which solves a non linear function using iteration.
def nonLinearSolver(f, x0, tol, maxIter):
  """
  Solves a non-linear function using iteration by finding the point where the function equals zero.

  Args:
    f: The function to solve.
    x0: The initial guess.
    tol: The tolerance for convergence.
    max_iter: The maximum number of iterations.

  Returns:
    The solution to the non-linear function.
  """

  if x0 == 0.0:
    x0 += 1e-9

  for i in range(maxIter):
    x1 = x0 - f(x0) / derivative(f, x0)  # Newton-Raphson method
    #print("iteration: " + str(i) + " - x1: " + str(x1))

    if abs((x1 - x0)/x0) < tol:
      return x1
    x0 = x1

  raise Exception("Non-linear solver did not converge.")

def derivative(f, x):
  """
  Calculates the derivative of a function using the central difference method.

  Args:
    f: The function to differentiate.
    x: The point at which to calculate the derivative.

  Returns:
    The derivative of the function at x.
  """

  h = 1e-6
  return (f(x + h) - f(x - h)) / (2 * h)




# find the number of expanding cells
if cellNumberHeight % 2 == 0: # if even
  totalHeightZero_f = lambda expansionRatio: sumOfGeometricSeries_f(firstCellHeight, expansionRatio, cellNumberHeight/2) * 2 - diameter

if cellNumberHeight % 2 == 1: # if odd
  totalHeightZero_f = lambda expansionRatio: sumOfGeometricSeries_f(firstCellHeight, expansionRatio, (cellNumberHeight - 1)/2) * 2 + firstCellHeight * expansionRatio ** ((cellNumberHeight+1)/2) - diameter

#totalHeightZero_f = lambda expansionRatio: sumOfGeometricSeries_f(firstCellHeight, expansionRatio, cellNumberHeight) - diameter


expansionRatio = nonLinearSolver(totalHeightZero_f, meshExpansionRatiox0, meshExpansionRatioTol, meshExpansionRatioMaxIter)
print("expansionRation = " + str(expansionRatio))

"""
the solver finds the expansion ratio at which the height of the domain is correct
print("x0=10 " + str(nonLinearSolver(totalHeightZero_f, 10.0)))
print("x0=2 " + str(nonLinearSolver(totalHeightZero_f, 2.0)))
print("x0=1 " + str(nonLinearSolver(totalHeightZero_f, 1.0)))
print("x0=0 " + str(nonLinearSolver(totalHeightZero_f, 0.0)))
print("x0=-1 " + str(nonLinearSolver(totalHeightZero_f, -1.0)))
"""


expansionRation = 1.06220818058417


'\nthe solver finds the expansion ratio at which the height of the domain is correct\nprint("x0=10 " + str(nonLinearSolver(totalHeightZero_f, 10.0)))\nprint("x0=2 " + str(nonLinearSolver(totalHeightZero_f, 2.0)))\nprint("x0=1 " + str(nonLinearSolver(totalHeightZero_f, 1.0)))\nprint("x0=0 " + str(nonLinearSolver(totalHeightZero_f, 0.0)))\nprint("x0=-1 " + str(nonLinearSolver(totalHeightZero_f, -1.0)))\n'

The expansion ratio of the domain is numerically determined. Therefore the result is not assumed to be exact, but rather approximate.

The mesh cell sizes are now generated by assuming the first cell height and incorporating the expansion ratio for the cells further from the boundary.

the height of the boundary cell, $h_{[0]}$, is
$$h_{[0]} = y_1, $$
the height of its first neighbouring cell, $h_{[1]}$ is
$$h_{[1]} = y_1 \cdot r_{\text{exp.}}, $$ and the height of the second neighbour is
$$h_{[2]} = h_{[1]} \cdot r_{\text{exp.}} = y_1 \cdot (r_{\text{exp.}})^2. $$ From this, the height of the $n$-th neighbouring cell can be written as
$$h_{[n]} = y_1 \cdot (r_{\text{exp.}})^n. $$

The height for each cell is determined and the total height is then checked to confirm whether this agrees with the required height of the domain.


In [38]:
def cellHeightForSymmetricPipe_f(firstCellHeight, expansionRatio, nthCell):
  """
  Arguments:
  firstCellHeight
  expansionRatio
  nthCell: the numebr of neighbours away from the nearest boundary cell. Boundary cells are nthCell = 0
  """
  cellHeight = firstCellHeight * expansionRatio ** nthCell
  return cellHeight

# prompt: Create an array with each element containing an object with the following variables: xVel, yVel, xPress, yPress

class Cell:
  def __init__(self, xVel, yVel, xPress, yPress, height, width):
    self.xVel = xVel
    self.yVel = yVel
    self.xPress = xPress
    self.yPress = yPress
    self.height = height
    self.width = width

# Initialise the mesh!

cell = []
for i in range(cellNumberHeight):
  cell.append(Cell(0.0, 0.0, 0.0, 0.0, -1.0, cellWidth))
  #print(str(i) + " "+ str(cell[i].height))


# Set cell heights
for i in range(cellNumberHeight):

  cellHeight = cellHeightForSymmetricPipe_f(firstCellHeight, expansionRatio, i)
  #print(str(firstCellHeight) + " " + str(expansionRatio) + " " + str(i) + " " + str(cellHeight) )

  # from the top
  if cell[i].height == -1.0:
    cell[i].height = cellHeight

  # in the symmetric case, the top  cell and the bottom cell are the same height.
  # from the bottom
  ii = cellNumberHeight - i - 1

  if cell[ii].height == -1.0:
    cell[ii].height = cellHeight

sumCellHeight = 0.0
for i in range(cellNumberHeight):
  sumCellHeight += cell[i].height
  print(sumCellHeight)

print(sumCellHeight)



8e-05
0.00016497665444673362
0.0002552395519587283
0.00035111754009919954
0.00045295992343996014
0.0005611377361547051
0.0006760450937780093
0.000798100629054794
0.0009277490171113743
0.001065462595504625
0.0012117430850514554
0.0013671234177079557
0.0015321696781575801
0.0017074831661819966
0.0018937025873282769
0.0020915063798535045
0.002301615186424375
0.0025247944795767307
0.0027618573505001557
0.003013667471307787
0.003281142241583541
0.0035652561306703185
0.003867044227875877
0.0041876060135305525
0.0045281093646356175
0.004889794809695742
0.005273980048236832
0.0056820647514748595
0.006115535661625555
0.006575972008432889
0.007065051262649931
0.007584555247433277
0.00813637662991622
0.008722525816610871
0.009345138277760685
0.010006482327327662
0.010708967386958368
0.011455152764036263
0.012247756975800685
0.013089667653502323
0.013983952062678166
0.014933868277873626
0.0159428770525238
0.017014654427238422
0.01815310512242532
0.019362376764044578
0.020646874994321003
0.02201127